### Install libraries

In [ ]:
pip install -U transformers praw torch numpy pandas

### Import libraries

In [5]:
import pandas as pd
import praw
from praw.models import MoreComments
from getpass import getpass

### Create authentication variables

In [22]:
client_id = getpass('Enter client_id: ')

In [23]:
client_secret = getpass('Enter client_secret: ')

In [24]:
password = getpass('Enter password: ')

### Create a Reddit instance

In [26]:
#Create a Reddit instance

reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    password=password,
    user_agent="testscript by u/curtpond",
    username="curtpond",
)

In [27]:
print(reddit.read_only)

False


In [28]:
reddit.read_only = True

### Inspect some Reddit data

In [37]:
# Get information about the Today I Learned subreddit
subreddit = reddit.subreddit('todayilearned')
 
# display the subreddit name
print(subreddit.display_name)
 
# display the subreddit title
print(subreddit.title)      
 
# display the subreddit description
print(subreddit.description)

todayilearned
Today I Learned (TIL)
[](http://www.reddit.com/r/aww/#newlink)
[New to reddit? Click here!](/wiki/reddit_101)



* You learn something new every day; what did *you* learn today?
 
* Submit interesting and **specific facts** that you just found out (not broad information you looked up, TodayILearned is not [/r/wikipedia](/r/wikipedia)).
 
#Posting rules#
 
1. **Submissions must be verifiable**. *Please link directly to a reliable source that supports every claim in your post title.* **Images alone do not count as valid references.** Videos are fine so long as they come from reputable sources (e.g. BBC, Discovery, etc).
1. **No personal opinions, anecdotes or subjective statements** (e.g "TIL xyz is a great movie").

1. **No recent sources.** Any sources (blog, article, press release, video, etc.) with a publication date more recent than two months are not allowed.
 
1. No politics, soapboxing, or agenda based submissions. This includes (but is not limited to) submissions r

In [38]:
# Look at the top 10 posts in the subreddit
ml_posts = reddit.subreddit('MachineLearning').hot(limit=10)
for post in ml_posts:
    print(post.title)

[D] Simple Questions Thread
[D] Machine Learning - WAYR (What Are You Reading) - Week 140
[P] Made a text generation model to extend stable diffusion prompts with suitable style cues
[P] Implementation of MagicMix from ByteDance researchers, - New way to interpolate concepts with much more natural, geometric coherency (implemented with Stable Diffusion!)
[N] eDiffi: Text-to-Image Diffusion Models with Ensemble of Expert Denoisers
[R] Code as Policies: Language Model Programs for Embodied Control - Robotics at Google 2022 - Robots that write their own code
[D] Graph neural networks
[D] What are the benefits of being a reviewer?
[R] ICML 2022 Paper Summaries (HUMAN)
[N] On the detection of synthetic images generated by diffusion models


### Create a dataframe with Reddit data

In [53]:
#Look at ML data
posts = []
ml_subreddit = reddit.subreddit('MachineLearning')
for post in ml_subreddit.hot(limit=10):
    posts.append(
    [   post.title, 
        post.score, 
        post.id, 
        post.subreddit, 
        post.url, 
        post.num_comments, 
        post.selftext, 
        post.created
    ]
        )
posts = pd.DataFrame(
    posts,
    columns=
    [ 
    
        'title', 
        'score', 
        'id', 
        'subreddit', 
        'url', 
        'num_comments', 
        'body', 
        'created'
    ]   
)
posts.head(10)

,title,score,id,subreddit,url,num_comments,body,created
0,[D] Simple Questions Thread,10,ybjvk5,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,79,Please post your questions here instead of cre...,1.666537e+09
1,[D] Machine Learning - WAYR (What Are You Read...,202,vg5kjd,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,57,This is a place to share machine learning rese...,1.655675e+09
2,[P] Made a text generation model to extend sta...,45,ykxr4v,MachineLearning,https://www.reddit.com/gallery/ykxr4v,5,,1.667469e+09
3,[P] Implementation of MagicMix from ByteDance ...,74,ykiuq0,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,6,Hi. Today I've came across this interesting pa...,1.667426e+09
4,[N] eDiffi: Text-to-Image Diffusion Models wit...,21,yks8g9,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,3,[https://arxiv.org/abs/2211.01324](https://ar...,1.667450e+09
5,[R] Code as Policies: Language Model Programs ...,44,ykm5rc,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,2,Paper: [https://arxiv.org/abs/2209.07753](http...,1.667433e+09
6,[D] Graph neural networks,58,ykajrg,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,24,I have never used GNNs before. I was wondering...,1.667408e+09
7,[D] What are the benefits of being a reviewer?,41,ykbp1s,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,34,So I just got invited to serve as a reviewer f...,1.667411e+09
8,[R] ICML 2022 Paper Summaries (HUMAN),3,ykqh58,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,0,"It's been a while since ICML 2022, but here ar...",1.667444e+09
9,[N] On the detection of synthetic images gener...,5,ykoodd,MachineLearning,https://www.reddit.com/r/MachineLearning/comme...,1,Paper: [https://arxiv.org/abs/2211.00680](http...,1.667439e+09


In [51]:
# create a list to store data for df conversion
books_list = []
# return 100 new posts from the books subreddit, currently in the top 25
books = reddit.subreddit("books").new(limit=100)
# return the important attributes
for post in books:
    books_list.append(
        [
            post.id,
            post.author,
            post.title,
            post.score,
            post.num_comments,
            post.permalink,
            post.selftext,
            post.created,
            post.pinned,
            post.total_awards_received,
        ]
    )

# create a dataframe
books_df = pd.DataFrame(
    books_list,
    columns=[
        "id",
        "author",
        "title",
        "score",
        "comments",
        "permalink",
        "post",
        "created",
        "pinned",
        "total awards",
    ],
)

# date converstion
books_df["created"] = pd.to_datetime(_posts["created"], unit="s")
books_df["created date"] = pd.to_datetime(_posts["created"], unit="s").dt.date
books_df["created time"] = pd.to_datetime(_posts["created"], unit="s").dt.time

In [52]:
books_df.head(10)

,id,author,title,score,comments,permalink,post,created,pinned,total awards,created date,created time
0,yl22q8,GrawliePie,The best book I've read in a VERY long time,1,0,/r/books/comments/yl22q8/the_best_book_ive_rea...,"""Song of the Crimson Flower"" by Julie C. Dao\n...",2022-11-03 12:13:21,False,0,2022-11-03,12:13:21
1,yl1zro,_Shoka_,I need friends,3,1,/r/books/comments/yl1zro/i_need_friends/,"Good day fellas! I am new to this subreddit, b...",2022-11-03 12:00:39,False,0,2022-11-03,12:00:39
2,yl1y21,GwenRat,Dani Shapiro: Signal Fires,1,0,/r/books/comments/yl1y21/dani_shapiro_signal_f...,Hello! I'm new to Reddit and wanted to share t...,2022-11-03 11:54:04,False,0,2022-11-03,11:54:04
3,yl15wc,oaklicious,Hemingway’s “The Sun Also Rises” - what am I m...,4,3,/r/books/comments/yl15wc/hemingways_the_sun_al...,Thought it would be fun to read some early 20t...,2022-11-03 11:18:06,False,0,2022-11-03,11:18:06
4,yl0tsd,Benelove67,Is there anyone here who loves reading Chinese...,3,4,/r/books/comments/yl0tsd/is_there_anyone_here_...,I've been drafting a male lead novel these day...,2022-11-03 11:16:18,False,0,2022-11-03,11:16:18
5,yl0npd,SAT0725,Emily Post's etiquette book overhauled for the...,2,0,/r/books/comments/yl0npd/emily_posts_etiquette...,,2022-11-03 11:00:14,False,0,2022-11-03,11:00:14
6,ykzr4h,TemporarySea911,It Starts With Us,1,3,/r/books/comments/ykzr4h/it_starts_with_us/,I have it ingrained in my mind that CoHo books...,2022-11-03 10:53:40,False,0,2022-11-03,10:53:40
7,ykzphq,snroberson,Can't decide on my next book. #firstworldproblems,0,4,/r/books/comments/ykzphq/cant_decide_on_my_nex...,I've been in a reading slump for years and yea...,2022-11-03 10:21:58,False,0,2022-11-03,10:21:58
8,ykzbhh,AutoModerator,AMA Requests: November 2022,1,1,/r/books/comments/ykzbhh/ama_requests_november...,"Welcome readers,\n\nOne of our more popular fe...",2022-11-03 10:14:46,False,0,2022-11-03,10:14:46
9,ykz5sr,tischawr,ballad of never after,1,0,/r/books/comments/ykz5sr/ballad_of_never_after/,"So! I've devoured the book, of course! And the...",2022-11-03 10:11:30,False,0,2022-11-03,10:11:30
